In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from IPython.core.display import display, HTML

from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from collections import Counter
pd.options.display.max_seq_items = None

In [3]:
df = pd.read_csv("../../data/features_model/df_story_features_model_whole.csv")


C:\Users\Justin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
from bs4 import BeautifulSoup
display(df.head())

# Generate binary dependent variable (for logit)

In [4]:
# Median split: Doesnt work if the median claps is 0
# df['isPopular'] = (df['ClapCount_Story'] > df['ClapCount_Story'].median()).astype(int)

# Below or above the top 25% of claps
threshold = 0.30
df['isPopular'] = (df['ClapCount_Story'] > df.sort_values(by=['ClapCount_Story'], ascending=False).iloc[int(len(df)*threshold)]['ClapCount_Story'])


In [ ]:
df[['isPopular', 'ClapCount_Story']].head()

# Split train / test (80/20)

In [5]:
mask = np.random.rand(len(df)) < 0.8
train = df[mask]
test = df[~mask]

In [ ]:
print("Length of dataset is", len(df))
print("Median clap is", train['ClapCount_Story'].median())

In [ ]:
xFeatures = ['isPaywall', 'isPublication', 'WordNum', 'SentimentPolarity', 'SentimentSubjectivity',  "TextSyllableNum",
          "TextLexiconNum", "TextSentenceNum", "ReadabilityFleschEase", "ReadabilitySMOG", "ReadabilityFleschKincaid", "ReadabilityColemanLiau", "ReadabilityARI", "ReadabilityDaleChall", "ReadabilityDifficultWordsNum", "ReadabilityLinsearWriteFormula", "ReadabilityGunningFog", "ReadabilityReadingTime", "ReadabilityConsensus", 
        'ReadingTime', 'HasFeaturedImage', 'CodeInlineNum', 'TagNum',
        'CodeBlockNum', 'CodeBlockLengthSum', 'CodeBlockLengthMedian', 'CodeBlockLengthMean',
        'CodeBlockLengthStd', 'CodeBlockLengthMin', 'CodeBlockLengthMax',
        'ListOlNum', 'ListOlSum', 'ListOlMedian', 'ListOlMean', 'ListOlStd', 'ListOlMin', 'ListOlMax', 
        'ListUlNum', 'ListUlSum', 'ListUlMedian', 'ListUlMean', 'ListUlStd', 'ListUlMin', 'ListUlMax', 
        'ImgNum', 'LinkNum',
        'HlightNum', 'ParagraphNum', 'ItalicNum', 'BoldNum', 
        'AuthorArticlesClap_mean','AuthorArticlesClap_median','AuthorArticlesClap_count','AuthorArticlesClap_sum',
        'AuthorArticlesResponse_mean','AuthorArticlesResponse_median','AuthorArticlesResponse_sum',
        'AuthorArticlesVoter_mean','AuthorArticlesVoter_median','AuthorArticlesVoter_sum',
        
        'TagUseSum', 'TagUseMean', 'TagUseMedian', 
        'TagClapSum','TagClapMean', 'TagClapMedian',  
        'PublicationClap_mean','PublicationClap_median','PublicationClap_count','PublicationClap_sum',
        'PublicationReadingTime_mean','PublicationReadingTime_median','PublicationReadingTime_sum',
        'PublicationVoter_mean','PublicationVoter_median','PublicationVoter_sum',
        'PublicationisPaywall_mean','PublicationisPaywall_median','PublicationisPaywall_sum',]

yFeatures = 'ClapCount_Story' # For linreg
# yFeatures = 'isPopular' # For logreg

xTrain = train[xFeatures]
yTrain = train[yFeatures]

xTest = test[xFeatures]
yTest = test[yFeatures]

In [6]:
features = ["isPaywall","isPublication",
"SentimentPolarity","SentimentSubjectivity",
"WordNum","TextSyllableNum","TextLexiconNum","TextSentenceNum",
"ReadabilityFleschEase","ReadabilitySMOG","ReadabilityFleschKincaid","ReadabilityColemanLiau","ReadabilityARI","ReadabilityDaleChall","ReadabilityDifficultWordsNum","ReadabilityLinsearWriteFormula","ReadabilityGunningFog","ReadabilityReadingTime","ReadabilityConsensus",
"ReadingTime","HasFeaturedImage","CodeInlineNum","TagNum","CodeBlockNum",
"CodeBlockLengthSum","CodeBlockLengthMedian","CodeBlockLengthMean","CodeBlockLengthStd","CodeBlockLengthMin","CodeBlockLengthMax",
"ListOlNum","ListOlSum","ListOlMedian","ListOlMean","ListOlStd","ListOlMin","ListOlMax",
"ListUlNum","ListUlSum","ListUlMedian","ListUlMean","ListUlStd","ListUlMin","ListUlMax","ImgNum","LinkNum","HlightNum",
"ParagraphNum","ItalicNum","BoldNum",
# "AuthorArticlesClap_mean","AuthorArticlesClap_median","AuthorArticlesClap_count","AuthorArticlesClap_sum",
"AuthorArticlesClap_count",
"AuthorArticlesResponse_median","AuthorArticlesResponse_sum",
"AuthorArticlesResponse_mean","AuthorArticlesVoter_mean","AuthorArticlesVoter_median","AuthorArticlesVoter_sum",
"TagUseSum","TagUseMean","TagUseMedian","TagClapSum","TagClapMean","TagClapMedian",
"PublicationClap_mean","PublicationClap_median",
"PublicationClap_count","PublicationClap_sum","PublicationReadingTime_mean","PublicationReadingTime_median","PublicationReadingTime_sum","PublicationVoter_mean","PublicationVoter_median","PublicationVoter_sum","PublicationisPaywall_mean","PublicationisPaywall_median","PublicationisPaywall_sum"

]
print(len(features))

depvar = 'ClapCount_Story'
depvar = 'isPopular'


xTrain = train[features]
yTrain = train[depvar]

xTest = test[features]
yTest = test[depvar]

76


# Check NA columns

In [ ]:
# pd.set_option('display.max_rows', 1000)
# display((df[df['TagNum']==0]))
# display(xTrain.isna().any())

# Generate model

In [ ]:
reg = LinearRegression(normalize=True).fit(xTrain, yTrain)

#sklearn preprocess normalize

In [ ]:
reg.score(xTrain,yTrain)

# statsmodel

In [ ]:
mod = sm.OLS(yTrain,xTrain.astype(float))
fii = mod.fit()

print(dir(fii))
print(p_values[p_values<0.05].sort_values(ascending=True))
print(fii.summary())

# Logistic Regression

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
logit = LogisticRegression()
logit.fit(xTrain,yTrain)

C:\Users\Justin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
logit.score(xTrain,yTrain)

0.7739138247041321

In [10]:
# Get AUC
from sklearn import metrics

yPreds = logit.predict(xTest)
print("Predict", Counter(yPreds))
# print(type(yPreds))
print("Truth", Counter(yTest))

# print(yPreds)
fpr, tpr, thresholds = metrics.roc_curve(yTest, yPreds)
print(metrics.auc(fpr, tpr))

Predict Counter({False: 1025607, True: 171903})
Truth Counter({False: 841708, True: 355802})
0.6540855139624928


# SVC